In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score

In [ ]:
# Want to upload this file to the session, otherwise need to mount Google Drive
matches = pd.read_csv("matches.csv", index_col=0)

In [ ]:
#matches["team"].value_counts()

Cleaning Data

In [ ]:
matches["date"] = pd.to_datetime(matches["date"])

Creating Predictors for Machine Learning

In [ ]:
matches["venue_code"] = matches["venue"].astype("category").cat.codes
matches["opp_code"] = matches["opponent"].astype("category").cat.codes
matches["hour"] = matches["time"].str.replace(":.+", "", regex=True).astype("int")
matches["day_code"] = matches["date"].dt.dayofweek

In [ ]:
matches["target"] = (matches["result"] == "W").astype("int")

Creating Initial ML Model

In [ ]:
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)

In [ ]:
train = matches[matches["date"] < '2022-01-01']
test = matches[matches["date"] > '2022-01-01']

In [ ]:
predictors = ['venue_code', 'opp_code', 'hour', 'day_code']

In [ ]:
rf.fit(train[predictors], train["target"])

RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [ ]:
preds = rf.predict(test[predictors])

In [ ]:
accuracy = accuracy_score(test["target"], preds)

In [ ]:
accuracy

0.6123188405797102

In [ ]:
combined = pd.DataFrame(dict(actual=test["target"], prediction=preds))

In [ ]:
pd.crosstab(index=combined["actual"], columns=combined["prediction"])

prediction,0,1
actual,,
0,141,31
1,76,28


Precision will tell us when we predicted a positive result, how often it was positive. #True Positives / (#True Positives + #False Positives). In other words, when we predicted a win, how often did the team actually win?

In [ ]:
precision_score(test["target"], preds)

0.4745762711864407

As we can see, the team only won ~47% of the time we predicted a win.

In [ ]:
grouped_matches = matches.groupby("team")

In [ ]:
group = grouped_matches.get_group("Manchester City")

In [ ]:
#group

In [ ]:
def rolling_averages(group, cols, new_cols):
  group = group.sort_values("date")
  # closed='left' tells the program not to include the current row in the
  # computations, which makes sense as including this would mean including
  # match data for a match we do not know yet (would be like seeing into
  # the future)
  rolling_stats = group[cols].rolling(3, closed='left').mean()
  group[new_cols] = rolling_stats
  # This will drop the rows with missing data, e.g. the first three rows whose
  # rolling hashes we cannot compute
  group = group.dropna(subset=new_cols)
  return group

In [ ]:
cols = ["gf", "ga", "sh", "sot", "dist", "fk", "pk", "pkatt"]
new_cols = [f"{c}_rolling" for c in cols]

In [ ]:
rolling_averages(group, cols, new_cols)

In [ ]:
matches_rolling = matches.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))

In [ ]:
matches_rolling = matches_rolling.droplevel('team')

In [ ]:
matches_rolling.index = range(matches_rolling.shape[0])

In [ ]:
def make_predictions(data, predictors):
    train = data[data["date"] < '2022-01-01']
    test = data[data["date"] > '2022-01-01']
    rf.fit(train[predictors], train['target'])
    preds = rf.predict(test[predictors])
    combined = pd.DataFrame(dict(actual=test["target"], prediction=preds, index=test.index))
    precision = precision_score(test["target"], preds)
    return combined, precision

In [ ]:
combined, precision = make_predictions(matches_rolling, predictors + new_cols)

In [ ]:
precision

0.5721784776902887

In [ ]:
combined = combined.merge(matches_rolling[["date", "team", "opponent", "result"]], left_index = True, right_index = True)

In [ ]:
combined

,actual,prediction,index,date,team,opponent,result
93,0,1,93,2022-01-23,Arsenal,Burnley,D
94,1,0,94,2022-02-10,Arsenal,Wolves,W
95,1,0,95,2022-02-19,Arsenal,Brentford,W
96,1,1,96,2022-02-24,Arsenal,Wolves,W
97,1,1,97,2022-03-06,Arsenal,Watford,W
...,...,...,...,...,...,...,...
3523,1,0,3523,2024-02-17,Wolverhampton Wanderers,Tottenham,W
3524,1,1,3524,2024-02-25,Wolverhampton Wanderers,Sheffield Utd,W
3525,0,0,3525,2024-03-02,Wolverhampton Wanderers,Newcastle Utd,L
3526,1,0,3526,2024-03-09,Wolverhampton Wanderers,Fulham,W


We want to combine rows so that for a given match, we combine the stats of the row for (teamA, teamB) with (teamB, teamA). However, because the team name for team vs. opponent team do not always align, we need to make a mapping class to replace some team names with new names.

In [ ]:
class MissingDict(dict):
  __missing__ = lambda self, key: key

map_values = {
    "Brighton and Hove Albion": "Brighton",
    "Manchester United": "Manchester Utd",
    "Newcastle United": "Newcastle Utd",
    "Tottenham Hotspur": "Tottenham",
    "West Ham United": "West Ham",
    "Wolverhampton Wanderers": "Wolves"
}
mapping = MissingDict(**map_values)

In [ ]:
combined["new_team"] = combined["team"].map(mapping)

In [ ]:
merged = combined.merge(combined, left_on=["date", "new_team"], right_on=["date", "opponent"])

In [ ]:
#merged

In [ ]:
merged[(merged["prediction_x"] == 1) & (merged["prediction_y"] == 0)]["actual_x"].value_counts()

1    193
0    129
Name: actual_x, dtype: int64